10/10/2023

This is a continuation of my Everett fire call data science project. I pruned and edited the fire call dataset, now I need to prune and edit my weather dataset.

Here's the explanation of the weather attributes, via a PDF NOAA sent along with my data download:

TEMP - Mean temperature for the day in degrees Fahrenheit to tenths. Missing = 9999.9

DEWP - Mean dew point for the day in degrees Fahrenheit to tenths. Missing = 9999.9

SLP - Mean sea level pressure for the day in millibars to tenths. Missing = 9999.9

STP - Mean station pressure for the day in millibars to tenths. Missing = 9999.9

VISIB - Mean visibility for the day in miles to tenths. Missing = 999.9

WDSP - Mean wind speed for the day in knots to tenths. Missing = 999.9 

MXSPD - Maximum sustained wind speed reported for the day in knots to tenths. Missing = 999.9

GUST - Maximum wind gust reported for the day in knots to tenths. Missing = 999.9

MAX - Maximum temperature reported during the day in Fahrenheit to tenths. Missing = 9999.9

MIN - Minimum temperature reported during the day in Fahrenheit to tenths. Missing = 9999.9

PRCP - Total precipitation (rain and/or melted snow) reported during the day in inches
and hundredths; will usually not end with the midnight observation (i.e. may include
latter part of previous day). “0” indicates no measurable precipitation (includes a trace). 
Missing = 99.99

SNDP - Snow depth in inches to tenths. It is the last report for the day if reported more than once. Missing = 999.9
 Note: Most stations do not report “0” on days with no snow on the ground, therefore, “999.9” will 
often appear on these days.

FRSHTT - Indicators (1 = yes, 0 = no/not reported) for the occurrence during the day of:
 Fog ('F' - 1st digit).
 Rain or Drizzle ('R' - 2nd digit).
 Snow or Ice Pellets ('S' - 3rd digit).
 Hail ('H' - 4th digit).
 Thunder ('T' - 5th digit).
 Tornado or Funnel Cloud ('T' - 6th digit).

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

filename = 'everett-weather-data.csv'
weatherData = pd.read_csv(filename)
weatherData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2835 entries, 0 to 2834
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  2835 non-null   int64  
 1   DATE     2835 non-null   object 
 2   DEWP     2835 non-null   float64
 3   FRSHTT   2835 non-null   int64  
 4   GUST     2835 non-null   float64
 5   MAX      2835 non-null   float64
 6   MIN      2835 non-null   float64
 7   MXSPD    2835 non-null   float64
 8   PRCP     2835 non-null   float64
 9   SLP      2835 non-null   float64
 10  SNDP     2835 non-null   float64
 11  STP      2835 non-null   float64
 12  TEMP     2835 non-null   float64
 13  VISIB    2835 non-null   float64
 14  WDSP     2835 non-null   float64
dtypes: float64(12), int64(2), object(1)
memory usage: 332.4+ KB


Just through scrolling through the csv, I can see that there's a ton of missing values in the dataset. I need to fill in all the missing values, since taking them out would totally ruin the plotting I'm hoping to do (number of fires against average temp, etc.). This requires some extra SQL work, contained in weather-averages.sql

In [10]:
weatherData["DEWP"] = weatherData["DEWP"].replace(9999.9, 43.2)
weatherData["GUST"] = weatherData["GUST"].replace(999.9, 22.73)
weatherData["MAX"] = weatherData["MAX"].replace(9999.9, 60.7)
weatherData["MXSPD"] = weatherData["MXSPD"].replace(999.9, 11.8)
weatherData["PRCP"] = weatherData["PRCP"].replace(99.99, 0.09)
weatherData["SLP"] = weatherData["SLP"].replace(9999.9, 1017.2)
weatherData["STP"] = weatherData["STP"].replace(9999.9, 784.5)
weatherData["TEMP"] = weatherData["TEMP"].replace(9999.9, 51.6) 
weatherData["VISIB"] = weatherData["VISIB"].replace(999.9, 9.2)
weatherData["WDSP"] = weatherData["WDSP"].replace(999.9, 6.1)


weatherData = weatherData.drop("MIN", axis=1) # min temp won't be nearly as interesting as avg or max
weatherData = weatherData.drop("STATION", axis=1)
weatherData = weatherData.drop("SNDP", axis=1) # all values in this column are 999.9 for whatever reason


In [11]:
# fog = np.zeros(len(bothData))
# rain = np.zeros(len(bothData))
# snow = np.zeros(len(bothData))
# hail = np.zeros(len(bothData))
# thunder = np.zeros(len(bothData))
# tornado = np.zeros(len(bothData))
# arr_arr = np.array([fog, rain, snow, hail, thunder, tornado])

# for index, row in bothData.iterrows():
#     string = str(row["FRSHTT"])
#     for k in range(len(string)):
#         print(string)
#         arr_arr[k][index] = int(string[k])
# bothData = bothData.drop("FRSHTT", axis=1)
# bothData["is_fog"] = fog
# bothData["is_rain"] = rain
# bothData["is_snow"] = snow
# bothData["is_hail"] = hail
# bothData["is_thunder"] = thunder
# bothData["is_tornado"] = tornado

# I tried to use this attribute, but it's very inconsistent in the dataset
# some entries are 0, some are 1000, some are 111000
weatherData = weatherData.drop("FRSHTT", axis=1)

In [12]:
weatherData["DATE"] = pd.to_datetime(weatherData["DATE"], format='%Y/%m/%d')
weatherData["DATE"] = weatherData["DATE"].dt.strftime('%m/%d/%Y') # editing dates to make them consistent with fire data's dates, for joining

Just doing fires by date wouldn't really work. I'd need to group by year and month to get counts of fires and average temperatures. I don't know how to do this in Python, but I (somewhat) remember how to do this in SQL. I think I'll need to make like 10 different csvs and scatter plot each.

In [13]:
weatherData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2835 entries, 0 to 2834
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    2835 non-null   object 
 1   DEWP    2835 non-null   float64
 2   GUST    2835 non-null   float64
 3   MAX     2835 non-null   float64
 4   MXSPD   2835 non-null   float64
 5   PRCP    2835 non-null   float64
 6   SLP     2835 non-null   float64
 7   STP     2835 non-null   float64
 8   TEMP    2835 non-null   float64
 9   VISIB   2835 non-null   float64
 10  WDSP    2835 non-null   float64
dtypes: float64(10), object(1)
memory usage: 243.8+ KB


In [14]:
# weatherData.to_csv(r'PrunedWeatherData.csv', index=False)